<p style="align: center;"><img align=center src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg"  width=400></p>

<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

In [1]:
from matplotlib import pyplot as plt
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
import torch

НИЖЕ ОБЪЯСНЕНИЕ КАК РАБОТАЕТ ПАТОРЧ НА КАГЛЕ

https://www.kaggle.com/aakashns/pytorch-basics-linear-regression-from-scratch

### 1. Нахождение сложной производной

Найдите производную по x от функции 
$$\sin\left(\tan(x)\frac{x^2}{y} + \ln(e^{-x^2 + 3}+x^3y)\right)\tan(x^2e^{x^9})$$

При этом надо пользоваться встроенным в PyTorch autograd. Численное вычисление производной может не дать нужный результат.

In [15]:
def find_x_derivative(x, y):
    # YOUR CODE HERE
    x = torch.tensor([x], dtype=float, requires_grad=True)
    y = torch.tensor([y], dtype=float, requires_grad=True)
    loss = torch.sin(torch.tan(x)*(x*x/y) + torch.log(torch.exp(-x*x+3) + x*x*x*y)) * torch.tan(x*x*(torch.exp(x**9)))
    loss.backward() 
                                                                                                
                                                                                                
    return x.grad

In [16]:
find_x_derivative(1, 21)

tensor([-9.0207], dtype=torch.float64)

# 2. Нахождение косинусной близости

Вам даны две матрицы A и B. Необходимо посчитать косинусную близость между строчками матрицы A и столбцами матрицы B. Ответ - матрица чисел, где номер строки - номер строки из матрицы А, а номер столбца - номер столбца из В, от которых бралась косинусная близость.

Напомним, что косинусная близость двух векторов - косинус угла между ними. В n-мерном пространстве косинус угла между веткорами удобнее всего через скалярное произведение:
$$\cos(angle(x, y)) = \frac{x \cdot y}{\left\|x\right\| \left\|y\right\|}$$

(Наша операция очень похожа на умножение матриц)

Ответом будет число, полученное в результате вызова torch.mean(get_cos_sim(A, B)), округлённое до 3х знаков после запятой

In [27]:
A = [[1, -47, 25, -3], [10, 17, -15, 22], [-3, -7, 26, 36], [12, -27, -42, 0]]

B = [[-50, -13, 1, 10, 1242], [21, 48, -13, -14, -20], [20, 15, 11, 43, 11], [11, 103, 147, 27, -8]]

In [67]:
torch.FloatTensor(B).norm(dim = 0, keepdim=True).shape

torch.Size([1, 5])

In [99]:
torch.FloatTensor(A)

tensor([[  1., -47.,  25.,  -3.],
        [ 10.,  17., -15.,  22.],
        [ -3.,  -7.,  26.,  36.],
        [ 12., -27., -42.,   0.]])

In [94]:
torch.FloatTensor(A)/torch.FloatTensor(A).norm(dim = 1, keepdim=True) # делим кажжый элемент строки на норму строки

tensor([[ 0.0188, -0.8813,  0.4688, -0.0563],
        [ 0.3018,  0.5130, -0.4527,  0.6639],
        [-0.0666, -0.1554,  0.5771,  0.7990],
        [ 0.2337, -0.5258, -0.8179,  0.0000]])

In [98]:
torch.FloatTensor(A).norm(dim = 1, keepdim=True)

tensor([[53.3292],
        [33.1361],
        [45.0555],
        [51.3517]])

In [101]:
torch.FloatTensor(A)/torch.FloatTensor(A).norm(dim = 1) # делим кажжый элемент строки на норму строки по подку - НЕ ВЕРНО

tensor([[ 0.0188, -1.4184,  0.5549, -0.0584],
        [ 0.1875,  0.5130, -0.3329,  0.4284],
        [-0.0563, -0.2113,  0.5771,  0.7010],
        [ 0.2250, -0.8148, -0.9322,  0.0000]])

In [97]:
torch.FloatTensor(A).norm(dim = 1)

tensor([53.3292, 33.1361, 45.0555, 51.3517])

Dim - 1 норма по строкам/ 0 по столбцам. keepdim - сохранить размерность.
например

по А - 4 строки / 4 нормы

по Б - 5 стоблцоы / 5 норм


In [75]:
def get_cos_sim(A, B):
    """
        A, B - torch float tensors
    """
    # YOUR CODE HERE
    A = torch.FloatTensor(A)
    B = torch.FloatTensor(B)
    norm_A = A.norm(dim = 1, keepdim=True)
    norm_B = B.norm(dim = 0, keepdim=True)
    
    res = torch.matmul(A/norm_A,B/norm_B)
    
    
    
    return torch.mean(res)# YOUR CODE HERE

In [76]:
get_cos_sim(A, B)

tensor(0.1498)

# 3. Линейная регрессия

Раньше мы самостоятельно считали производные, чтобы находить веса линейной регрессии с помощью градиентного спуска. Теперь нам нужно использовать для этого PyTorch и его autograd. 

**Важно**: на самом деле .backward не обновляет содержимое матриц с производными (some_tensor.grad), а прибавляет к ним только что посчитаные значения проивзодных. Это значит, что вызвав .backward дважды, вы получите удвоенную производную. Так как мы обновляем веса в цикле и много раз вызываем .backward, то очень быстро мы получим мусор в some_tensor.grad, если не будем его каждый раз обнулять. Таким образом, в конц итериации после использования производных обнулите значения в матрице производных для всех нужных Вам переменных. Делается это вот так 
> some\_tensor.grad.data.zero_()

In [240]:
class LinearRegression:
    def get_loss(self, preds, y):
        """
            @param preds: предсказания модели
            @param y: истиные значения
            @return mse: значение MSE на переданных данных
        """
        # возьмите средний квадрат ошибки по всем выходным переменным
        # то есть сумму квадратов ошибки надо поделить на (количество_элементов * количество_таргетов) 
        # between each element!!! т.е. в n раз меньше чем n строк
        diff = preds - y
        
        return torch.sum(diff * diff) / diff.numel() #YOUR_CODE
    
    def init_weights(self, input_size, output_size):
        """
            Инициализирует параметры модели
            W - матрица размерности (input_size, output_size)
            инициализируется рандомными числами из
            uniform распределения (torch.rand())
            b - вектор размерности (1, output_size)
            инициализируется нулями
        """
        torch.manual_seed(0) #необходимо для воспроизводимости результатов
        self.W = torch.rand(input_size, output_size,  requires_grad=True)
        self.b = torch.zeros(1, output_size, requires_grad=True)

    def fit(self, X, y, num_epochs=1000, lr=0.001):
        """
            Обучение модели линейной регрессии методом градиентного спуска
            @param X: размерности (num_samples, input_shape)
            @param y: размерности (num_samples, output_shape)
            @param num_epochs: количество итераций градиентного спуска
            @param lr: шаг градиентного спуска
            @return metrics: вектор значений MSE на каждом шаге градиентного
            спуска.
        """
        self.init_weights(X.shape[1], y.shape[1])
        metrics = []
        for _ in range(num_epochs):
            preds = self.predict(X)
            # сделайте вычисления градиентов c помощью Pytorch и обновите веса
            # осторожнее, оберните вычитание градиента в 
            #                 with torch.no_grad():
            #                     #some code
            # иначе во время прибавления градиента к переменной создастся очень много нод в дереве операций
            # и ваши модели в будущем будут падать от нехватки памяти
            loss = self.get_loss(preds, y)
            loss.backward()
            self.W.data -=  lr * self.W.grad # YOUR_CODE
            self.b.data -=  lr * self.b.grad # YOUR_CODE
            self.W.grad.zero_()
            self.b.grad.zero_()
            '''
            loss = self.get_loss(preds, y)
            loss.backward()
            with torch.no_grad():
                self.W -=  lr * self.W.grad
                self.b -=  lr * self.b.grad
                self.W.grad.zero_()
                self.b.grad.zero_()
            
            #YOUR_CODE
            #YOUR_CODE
            #YOUR_CODE
            #YOUR_CODE'''
            metrics.append(self.get_loss(preds, y).data)
        return metrics

    def predict(self, X):
        """
            Думаю, тут все понятно. Сделайте свои предсказания :)
        """
        preds =  X.mm(self.W) + self.b
        return preds #YOUR_CODE

**Вот эта хрень torch.no_grad() не обязательна, ее суть в том, чтобы позволить вычислить градиент и изменить ту переменную по которой вычислялся градиент (заморочки pytorch), короче она временно сбрасывает requires_grad в false. Однако есть и другой вариант, можно обновить self.W.data и не трогать сам self.W на котором установлен requires_grad=True. Работает также **

1. Сгенерируйте данные с помощью make_regression с параметрами n_targets=3, n_features=2, noise=10, random_state=42. 
2. Обучите модель линейной регрессии, оставив в fit параметры num_epochs и lr по умолчанию (обратите внимание, что перед обучением нужно привести данные к типу, использующимся в torch) 
2. Посчитайте среднее значение метрики MSE по всем итерациям цикла в fit (массив из значений MSE на каждой итерации возвращается из метода fit). Ответом, который необходимо сдать в систему, будет число, округленное до 3х знаков после запятой.

In [241]:
X, Y = datasets.make_regression(n_targets=3, n_features=2, noise=10, random_state=42) # YOUR CODE HERE
X = torch.FloatTensor(X) # YOUR CODE HERE
Y = torch.FloatTensor(Y) # YOUR CODE HERE
model = LinearRegression()
mse = model.fit(X, Y)

In [242]:
print(np.mean(mse))

4256.561


Здесь предлагаем протестировать метод predict удобным вам образом.

In [0]:
#YOUR CODE